In [1]:
!pip install mistralai tqdm psycopg2-binary sqlalchemy python-dotenv pandas termcolor openpyxl black --quiet

%load_ext autoreload
%autoreload 2

from database.getdata import query_data, get_connection, execute_query
from database.matching import find_matches, compare_names

import pandas as pd 
import os
from mistralai import Mistral
from dotenv import load_dotenv
import tqdm

import json
load_dotenv()

True

## LOAD DATASET Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.xlsx


In [2]:
# read csv file data/Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.csv

df = pd.read_csv('../database/data/Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.csv')
df

,Film id,Titre,Poste,Nom,Genre,Visa,Genre_Film,Bonus_Parité,Devis,Tranche_Devis,Femme_Real
0,53337,Rue des Dames,Producteur délégué,Hamé,H,147799,fiction,0,2038083,3_ De 2 à 3 millions,Non
1,56951,Vourdalak,Producteur délégué,Judith Lou Lévy,F,156774,Fiction,NaN,1797405,2_ De 1 à 2 millions,Non
2,56353,Low-Tech,Producteur délégué,Lissandra Haulica,F,154656,documentaire,0,207079,1_ Moins de 1 million,Non
3,54224,Polaris,Producteur délégué,Clara Vuillermoz,F,155317,documentaire,x,463611,1_ Moins de 1 million,Oui
4,52192,Pulse,Producteur délégué,Sébastien Aubert,H,148404,fiction,x,1219975,2_ De 1 à 2 millions,Oui
...,...,...,...,...,...,...,...,...,...,...,...
3512,56429,Soudain seuls,Effets spéciaux,Pierre Procoudine-Gorsky,H,146672,fiction,0,14333978,15_ De 14 à 15 millions,Non
3513,55883,Règne animal,Effets spéciaux,Pascal Molina,H,155459,fiction,0,15983936,16_ De 15 à 16 millions,Non
3514,54622,Guerre des Lulus,Effets spéciaux,Martin Trepreau,H,152873,fiction,0,7987433,8_ De 7 à 8 millions,Non
3515,55708,Alibi.com 2,Effets spéciaux,Stephane Bidault,H,157074,fiction,0,18433224,19_ De 18 à 19 millions,Non


In [3]:
print(f"Nombre de lignes : {df.shape[0]}")
print(f"Nombre de Visa uniques : {df['Visa'].nunique()}")
print(f"Nombre de Titre uniques : {df['Titre'].nunique()}")

Nombre de lignes : 3517
Nombre de Visa uniques : 247
Nombre de Titre uniques : 247


In [4]:
print(f"NOmbre de cheffes de postes distincts : {df['Nom'].nunique()}")
print(f"Nombre de Postes distincts : {df['Poste'].nunique()}")

NOmbre de cheffes de postes distincts : 2445
Nombre de Postes distincts : 13


In [5]:
df['Femme_Real'].unique()


array(['Non', 'Oui'], dtype=object)

In [6]:
df['Bonus_Parité'].unique()

array(['0', nan, 'x'], dtype=object)

In [7]:
df_cp = query_data('SELECT * FROM inegalites_cinema.cheffes_postes')
df_cp


,id,type_de_poste,nom,genre,nom2,created_at,updated_at


In [8]:
df_film = query_data('SELECT * FROM inegalites_cinema.film')
df_film

,id,nom_originel,date_sortie_france,pays,duree,langue_principale,format,n_visa_exploitation,devis,genre,bonus_parite,femme_realisatrice,created_at,updated_at


## J'AJOUTE LES FILMS

In [9]:
df_film = pd.DataFrame({
    'nom_originel': df['Titre'],
    'n_visa_exploitation': df['Visa'].where(df['Visa'].notna(), None),
    'genre': df['Genre_Film'].where(df['Genre_Film'].notna(), None),
    'bonus_parite': df['Bonus_Parité'] == 'x',
    'devis': df['Devis'].where(df['Devis'].notna(), None),
    'femme_realisatrice': df['Femme_Real'] == 'Oui'
})

df_film.drop_duplicates().to_sql(
    'film',
    get_connection(),
    schema='inegalites_cinema',
    if_exists='append',
    index=False
)

247

## J'AJOUTE LES CHEFFES DE POSTES QUI NE SONT PAS DANS LA BASE DE DONNEES

In [10]:
# Create new DataFrame directly from source columns
new_cheffes_postes = pd.DataFrame({
    'nom': df['Nom'],
    'type_de_poste': df['Poste'],
    'genre': df['Genre']
})

# Remove duplicates and append to database
new_cheffes_postes.drop_duplicates().to_sql(
    'cheffes_postes',
    get_connection(),
    schema='inegalites_cinema',
    if_exists='append',
    index=False
)

755

In [11]:
df_cp = query_data('SELECT * FROM inegalites_cinema.cheffes_postes')

df_film = query_data('SELECT * FROM inegalites_cinema.film')

In [12]:
# join df with df_film
df_merged = df.merge(df_film, left_on='Titre', right_on='nom_originel', how='left')
df_merged.rename(columns={'id': 'id_film'}, inplace=True)

# merge with df_cp on Nom<->nom and Poste<->type_de_poste
df_merged = df_merged.merge(df_cp,
                            left_on=['Nom', 'Poste'],
                            right_on=['nom', 'type_de_poste'],
                            how='left')

df_merged.rename(columns={'id': 'id_cheffes_postes'}, inplace=True)

df_merged[['id_film', 'id_cheffes_postes']].to_sql(
    'film_cheffes_postes',
    get_connection(),
    schema='inegalites_cinema',
    if_exists='replace',
    index=False
)

517

In [13]:
df[df['Nom']=='Julie Lerat-Gersant']

,Film id,Titre,Poste,Nom,Genre,Visa,Genre_Film,Bonus_Parité,Devis,Tranche_Devis,Femme_Real
1148,54589,Petites,Scénariste,Julie Lerat-Gersant,F,155500,fiction,x,1851787,2_ De 1 à 2 millions,Oui
3042,54589,Petites,Réalisation,Julie Lerat-Gersant,F,155500,fiction,x,1851787,2_ De 1 à 2 millions,Oui


In [14]:
df_merged[df_merged['Nom']=='Julie Lerat-Gersant']

,Film id,Titre,Poste,Nom,Genre,Visa,Genre_Film,Bonus_Parité,Devis,Tranche_Devis,...,femme_realisatrice,created_at_x,updated_at_x,id_cheffes_postes,type_de_poste,nom,genre_y,nom2,created_at_y,updated_at_y
1148,54589,Petites,Scénariste,Julie Lerat-Gersant,F,155500,fiction,x,1851787,2_ De 1 à 2 millions,...,True,2025-02-21 06:58:02.337180,2025-02-21 06:58:02.337180,895,Scénariste,Julie Lerat-Gersant,F,None,2025-02-21 06:58:02.399258,2025-02-21 06:58:02.399258
3042,54589,Petites,Réalisation,Julie Lerat-Gersant,F,155500,fiction,x,1851787,2_ De 1 à 2 millions,...,True,2025-02-21 06:58:02.337180,2025-02-21 06:58:02.337180,2430,Réalisation,Julie Lerat-Gersant,F,None,2025-02-21 06:58:02.399258,2025-02-21 06:58:02.399258


In [15]:
df_film[df_film['nom_originel']=='Petites']

,id,nom_originel,date_sortie_france,pays,duree,langue_principale,format,n_visa_exploitation,devis,genre,bonus_parite,femme_realisatrice,created_at,updated_at
112,113,Petites,None,None,None,None,None,155500,1851787,fiction,True,True,2025-02-21 06:58:02.337180,2025-02-21 06:58:02.337180


In [16]:
find_matches('Petites', df_film, 'nom_originel')

,id,nom_originel,date_sortie_france,pays,duree,langue_principale,format,n_visa_exploitation,devis,genre,bonus_parite,femme_realisatrice,created_at,updated_at,similarity_score
112,113,Petites,None,None,None,None,None,155500,1851787,fiction,True,True,2025-02-21 06:58:02.337180,2025-02-21 06:58:02.337180,1.000000
90,91,Petite,None,None,None,None,None,156175,7430853,fiction,False,False,2025-02-21 06:58:02.337180,2025-02-21 06:58:02.337180,0.914286
